## Robust Mean - Variance Portfolio Optimization (WORK IN PROGRESS)

Robust portfolio optimization is a variant of traditional portfolio optimization. The objective is to allocate a total investment amount across different assets to maximize the portfolio's expected return, while ensuring a very low probability (e.g., 0.5%) that the actual return falls below this expected value. This approach addresses the uncertainty inherent in portfolio returns, making it a robust optimization problem.


Consider an example where we have n = 200 assets. Let $r_i$ denote the return of the $i$-th asset. The return on Asset #200 $r_{n}=1.05$ has zero variability. The returns of the remaining assets $r_i$, $\forall i \in [n - 1]$, are random variables taking values in the intervals $[\mu_i - \sigma_i ,\mu_i + \sigma_i  ]$. $\vec{\mu}$ and $\vec{\sigma}$ are defined as:

$$ \mu_i = 1.05 + \frac{0.3\left(n - i\right)}{n - 1} , \space \sigma_i = 0.05 + \frac{0.6\left(n - i\right)}{n - 1}, \space \forall i \in [n - 1] $$



To solve this problem, we first import the required packages and generate the data. 

In [6]:
import lropt
import cvxpy as cp
import numpy as np
import warnings
from scipy.sparse import SparseEfficiencyWarning
warnings.filterwarnings('ignore', category=UserWarning, module='cvxpy')
warnings.filterwarnings('ignore', category=SparseEfficiencyWarning)


In [7]:
N = 200
GUARENTEED_RETURN = 1.05
b2 = 0.3
b3 = 0.05
b4 = 0.6

mu = np.zeros(N - 1)
for i in range(N - 1):
    mu_i = GUARENTEED_RETURN + b2 * (N - i) / (N- 1)
    mu[i] = mu_i  

sigma = np.zeros(N - 1)
for i in range(N - 1):
    sigma_i = b3 + b4 * (N - i) / (N - 1)
    sigma[i] = sigma_i 


The problem we want to solve is the uncertain linear optimization problem:

$$ 
\begin{aligned}
& \text{maximize} \quad t\\
& \text{subject to} \\
& \mu^T \vec{x} + \sigma^T \vec{x} + 1.05x_{n} \geq t , \\
& 1^T\vec{x}= 1, \\
& \vec{x} \geq 0
\end{aligned}
$$

$ x_i $ is the capital to be invested in asset $i$. The guarenteed yearly return is 1.05.



In this problem, the uncertain data are the retuns $r_i = \mu_i + \sigma_i z_i, i \in [199]$ where $z_i, i\in[199]$, are independant random variables with zero mean varying in the segments $[-1, 1]$. The robust counterpart to the uncertain linear optimization problem is:


$$ 
\begin{aligned}
& \text{maximize} \quad t\\
& \text{subject to} \\
& (\vec{\mu} + \sigma^T \vec{z})\vec{x} + 1.05x_{n - 1} \geq t , \quad \forall z \in \mathcal{Z} \\
& 1^T\vec{x}= 1, \\
& \vec{x} \geq 0
\end{aligned}
$$

for a variety of uncertainty sets $ \mathcal{Z} $. 


In the following snippet, we solve this problem using three uncertainty sets and compare the results. (ADD FORMULATIONS FOR UNCERTAINTY SETS)

In [9]:
uncertainty_sets = [lropt.Ellipsoidal(p = 2, rho = 2), lropt.Budget(), lropt.Box(rho = 1, a = np.diag(sigma))]
names = ['ellipsoidal', 'budget', 'box']
num_dec = 3

for uc in enumerate(uncertainty_sets):
    t = cp.Variable()
    x = cp.Variable(N, nonneg=True)
    z = lropt.UncertainParameter(N - 1, uncertainty_set=uc[1])

    constraints = [
    (mu + sigma @ z) @ x[:(N-1)] + GUARENTEED_RETURN * x[-1] >=t,
    cp.sum(x) == 1
    ]
    objective = cp.Maximize(t)
    prob = lropt.RobustProblem(objective, constraints)
    prob.solve()
    
    print(f"The robust optimal value using {names[uc[0]]} uncertainty is {round(float(t.value), num_dec)}")

The robust optimal value using ellipsoidal uncertainty is 1.05
The robust optimal value using budget uncertainty is 1.05
The robust optimal value using box uncertainty is 1.05


The value $t$ is the guarenteed return on the entire portfolio. Looking at the results without rounding, there is a very slight difference in the three values.